In [1]:
import os
import cv2
import numpy as np
import dlib
from skimage.feature import local_binary_pattern
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Load the Dlib facial landmark predictor
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Paths to datasets
down_syndrome_path = 'downSyndrome'
healthy_path = 'healty'

# Constants for LBP
RADIUS = 2
POINTS = 8 * RADIUS
METHOD = 'uniform'

# Size of patches around landmarks
PATCH_SIZE = 16

# List of specific landmarks to extract patches from
landmark_indices = [
    36, 39, 42, 45, 27, 30, 33, 31, 35, 51, 48, 54, 57, 68
]

# Define landmark pairs for geometric features
pairs = [
    (36, 39), (39, 42), (42, 45), (27, 30), (30, 33), (33, 31),
    (33, 35), (30, 31), (30, 35), (33, 51), (51, 48), (51, 54),
    (51, 57), (48, 57), (54, 57), (39, 68), (42, 68)
]

In [2]:
# Check if the face is frontal by verifying symmetry
def is_frontal_face(landmarks):
    left_eye = np.mean(np.array(landmarks[36:42]), axis=0)
    right_eye = np.mean(np.array(landmarks[42:48]), axis=0)
    nose_tip = np.array(landmarks[30])
    eye_distance = np.linalg.norm(left_eye - right_eye)
    nose_to_left_eye = np.linalg.norm(nose_tip - left_eye)
    nose_to_right_eye = np.linalg.norm(nose_tip - right_eye)
    symmetry_threshold = 0.15 * eye_distance
    return abs(nose_to_left_eye - nose_to_right_eye) < symmetry_threshold

# Modified function to get landmarks and add the 69th point
def get_landmarks(image_input):
    if isinstance(image_input, str):
        img = cv2.imread(image_input)
    else:
        img = image_input
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)
    for face in faces:
        landmarks = predictor(gray, face)
        points = [(landmarks.part(n).x, landmarks.part(n).y) for n in range(68)]
        midpoint_x = (points[21][0] + points[22][0]) // 2
        midpoint_y = (points[21][1] + points[22][1]) // 2
        points.append((midpoint_x, midpoint_y))
        if is_frontal_face(points):
            return points
    return None

# Improved data augmentation function
def augment_image(image):
    augmented_images = []
    # Flip horizontally
    augmented_images.append(cv2.flip(image, 1))
    # Rotation
    rows, cols = image.shape[:2]
    for angle in [-15, -10, 0, 10, 15]:
        M = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
        augmented_images.append(cv2.warpAffine(image, M, (cols, rows)))
    # Add Gaussian noise
    noise = np.random.normal(0, 10, image.shape).astype(np.uint8)
    noisy_image = cv2.add(image, noise)
    augmented_images.append(noisy_image)
    return augmented_images

# Function to extract patches around specific landmarks
def extract_patches(image, landmarks, indices, patch_size=PATCH_SIZE):
    patches = []
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    for idx in indices:
        (x, y) = landmarks[idx]
        x_start = max(x - patch_size // 2, 0)
        y_start = max(y - patch_size // 2, 0)
        x_end = min(x + patch_size // 2, gray.shape[1])
        y_end = min(y + patch_size // 2, gray.shape[0])
        patch = gray[y_start:y_end, x_start:x_end]
        if patch.size > 0:
            patches.append(patch)
    return patches

# Function to extract LBP features from patches
def extract_lbp_from_patches(patches, radius=RADIUS, points=POINTS, method=METHOD):
    lbp_features = []
    for patch in patches:
        lbp = local_binary_pattern(patch, points, radius, method)
        hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, points + 3), range=(0, points + 2))
        hist = hist.astype("float")
        hist /= (hist.sum() + 1e-6)
        lbp_features.extend(hist)
    return lbp_features

# Function to extract geometric features from specific landmark pairs
def extract_geometric_features(landmarks, pairs):
    geom_features = []
    for i, j in pairs:
        p1 = landmarks[i]
        p2 = landmarks[j]
        distance = np.linalg.norm(np.array(p1) - np.array(p2))
        geom_features.append(distance)
    return geom_features

# Function to extract combined features (LBP + Geometric)
def get_combined_features(image, pairs):
    landmarks = get_landmarks(image)
    if landmarks:
        patches = extract_patches(image, landmarks, landmark_indices)
        lbp_features = extract_lbp_from_patches(patches)
        geom_features = extract_geometric_features(landmarks, pairs)
        combined_features = lbp_features + geom_features
        return combined_features
    return None

In [3]:
# Collect data and labels
X = []
y = []

for path, label in [(down_syndrome_path, 1), (healthy_path, 0)]:
    for img_file in os.listdir(path):
        img_path = os.path.join(path, img_file)
        img = cv2.imread(img_path)
        if img is not None:
            combined_features = get_combined_features(img, pairs)
            if combined_features:
                X.append(combined_features)
                y.append(label)
                
                # Apply data augmentation and extract features
                augmented_images = augment_image(img)
                for aug_img in augmented_images:
                    aug_features = get_combined_features(aug_img, pairs)
                    if aug_features:
                        X.append(aug_features)
                        y.append(label)

X = np.array(X)
y = np.array(y)

# Handle NaNs and scale features
X = np.nan_to_num(X)
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Reduce dimensionality using PCA
pca = PCA(n_components=50)
X = pca.fit_transform(X)

In [4]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [5]:
from sklearn.ensemble import RandomForestClassifier

# Train the Random Forest classifier
model = RandomForestClassifier(
    n_estimators=100,  
    max_depth=None,    
    random_state=42,   
    class_weight="balanced"  
)

# Measure training time
import time

start_time = time.time() 
model.fit(X_train, y_train)  
end_time = time.time() 

training_time = end_time - start_time  
print(f"Training Time: {training_time:.2f} seconds")

Training Time: 5.06 seconds


In [7]:
cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
print(f"5-Fold Cross-Validation Accuracy: {np.mean(cv_scores):.2f}")
# Evaluate on the test set
y_pred_test = model.predict(X_test)
y_prob_test = model.predict_proba(X_test)[:, 1]  # Probabilities for AUC calculation

accuracy_test = accuracy_score(y_test, y_pred_test)
precision_test = precision_score(y_test, y_pred_test)
recall_test = recall_score(y_test, y_pred_test)
auc_test = roc_auc_score(y_test, y_prob_test)

print(f"Test Accuracy: {accuracy_test:.2f}")
print(f"Test Precision: {precision_test:.2f}")
print(f"Test Recall: {recall_test:.2f}")
print(f"Test AUC: {auc_test:.2f}")
print("Test Classification Report:")
print(classification_report(y_test, y_pred_test))

# Evaluate on the training set
y_pred_train = model.predict(X_train)
y_prob_train = model.predict_proba(X_train)[:, 1]  # Probabilities for AUC calculation

accuracy_train = accuracy_score(y_train, y_pred_train)
precision_train = precision_score(y_train, y_pred_train)
recall_train = recall_score(y_train, y_pred_train)
auc_train = roc_auc_score(y_train, y_prob_train)

print(f"Training Accuracy: {accuracy_train:.2f}")
print(f"Training Precision: {precision_train:.2f}")
print(f"Training Recall: {recall_train:.2f}")
print(f"Training AUC: {auc_train:.2f}")
print("Training Classification Report:")
print(classification_report(y_train, y_pred_train))

5-Fold Cross-Validation Accuracy: 0.89
Test Accuracy: 0.91
Test Precision: 0.93
Test Recall: 0.85
Test AUC: 0.97
Test Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.95      0.92      1454
           1       0.93      0.85      0.89      1196

    accuracy                           0.91      2650
   macro avg       0.91      0.90      0.90      2650
weighted avg       0.91      0.91      0.90      2650

Training Accuracy: 1.00
Training Precision: 1.00
Training Recall: 1.00
Training AUC: 1.00
Training Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3365
           1       1.00      1.00      1.00      2818

    accuracy                           1.00      6183
   macro avg       1.00      1.00      1.00      6183
weighted avg       1.00      1.00      1.00      6183



In [9]:
import joblib

model_data = {
    'model': model,
    'scaler': scaler,
    'pca': pca 
}
joblib.dump(model_data, 'rf_model.pkl')


print("Model and scaler saved to 'rf_model.pkl'")


Model and scaler saved to 'rf_model.pkl'
